In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132164")
exp = Experiment(workspace=ws, name="udacity-project-one")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RN69XCBR5 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-132164
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-132164


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_target = ComputeTarget(ws, name='projectonecompute')
# ami = AmlCompute(compute_target, name='aml-quickstarts-132164')

In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps =  RandomParameterSampling({
    "C": uniform(0.4, 1.0)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
                  entry_script='train.py',
                  compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                          hyperparameter_sampling=ps,
                          policy=policy,
                          primary_metric_name='Accuracy',
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs=10,
                          max_concurrent_runs=8)

In [26]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a29e4298-ce4f-497d-a266-b48def806ddf
Web View: https://ml.azure.com/experiments/udacity-project-one/runs/HD_a29e4298-ce4f-497d-a266-b48def806ddf?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-132164/workspaces/quick-starts-ws-132164

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-28T16:37:03.771495][API][INFO]Experiment created<END>\n""<START>[2020-12-28T16:37:04.383131][GENERATOR][INFO]Trying to sample '8' jobs from the hyperparameter space<END>\n""<START>[2020-12-28T16:37:04.728339][GENERATOR][INFO]Successfully sampled '8' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-28T16:37:05.8380933Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_a29e4298-ce4f-497d-a266-b48def806ddf
Web View: https://ml.azure.com/experiments/udacity-project-one/runs/HD_a29e4298-ce4f-497d-a266-b48def806ddf?wsid=/subscriptio

{'runId': 'HD_a29e4298-ce4f-497d-a266-b48def806ddf',
 'target': 'projectonecompute',
 'status': 'Canceled',
 'startTimeUtc': '2020-12-28T16:37:03.489076Z',
 'endTimeUtc': '2020-12-28T16:42:58.546975Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name

In [24]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

### YOUR CODE HERE ###
best_run.register_model(model_name='hyperdrive-best-model', model_path='./outputs/model.joblib')
# best_run

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_1e5b5c4300e6e6b6927ec6eb8e6499fafee3fed097e03a7f3b96304dfa4addd8_d.txt', 'azureml-logs/65_job_prep-tvmps_1e5b5c4300e6e6b6927ec6eb8e6499fafee3fed097e03a7f3b96304dfa4addd8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_1e5b5c4300e6e6b6927ec6eb8e6499fafee3fed097e03a7f3b96304dfa4addd8_d.txt', 'logs/azureml/100_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_bd1073a1-1b75-408e-b989-e1cb226349a9.jsonl', 'logs/azureml/dataprep/python_span_l_bd1073a1-1b75-408e-b989-e1cb226349a9.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorRe

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = TabularDatasetFactory.from_delimited_files(path,
                                                validate=True,
                                                include_path=False,
                                                infer_column_types=True,
                                                separator=',',
                                                header=True,
                                                support_multi_line=False,
                                                empty_as_string=False,
                                                encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= "validation_acc",
    training_data= x,
    label_column_name= y,
    n_cross_validations=7)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, name="automl_job_classification_experiment")
auto_ml_run = experiment.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run = auto_ml_run.get_best_run_by_primary_metric()

### YOUR CODE HERE ###
best_run.register_model(model_name='automl_job_classification-best-model', model_path='outputs/model-automl.joblib')